In [1]:
import pyspark
from pyspark.sql.functions import col, expr
import pyspark.sql.functions as f
import pandas as pd

spark = pyspark.sql.SparkSession.builder.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/21 11:04:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/21 11:04:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Create a spark data frame that contains your favorite programming languages.


### The name of the column should be language  


In [2]:
lang_df = spark.createDataFrame(
    pd.DataFrame(
        {
            "Lanuage": [
                "Python",
                "SQL",
                "VBA",
                "NotJava",
                "NotC",
            ]
        }
    )
)

/usr/local/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


### View the schema of the dataframe  


In [3]:
lang_df.printSchema(), lang_df.show()

root
 |-- Lanuage: string (nullable = true)



+-------+
|Lanuage|
+-------+
| Python|
|    SQL|
|    VBA|
|NotJava|
|   NotC|
+-------+



(None, None)

### Output the shape of the dataframe  


In [4]:
print(lang_df.count(), "rows", len(lang_df.columns), "columns")

5 rows 1 columns


### Show the first 5 records in the dataframe  

In [5]:
lang_df.show(5)

+-------+
|Lanuage|
+-------+
| Python|
|    SQL|
|    VBA|
|NotJava|
|   NotC|
+-------+



# Load the mpg dataset as a spark dataframe.


In [6]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



/usr/local/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():



### Create 1 column of output that contains a message like the one below:
    The 1999 audi a4 has a 4 cylinder engine.
For each vehicle.

In [7]:
mpg.select(f.concat(\
            f.lit("The "), \
            mpg.year, \
            f.lit(" "), \
            mpg.manufacturer, \
            f.lit(" "), \
            mpg.model, \
            f.lit(" has a "), \
            mpg.cyl, \
            f.lit(" cylinder engine")\
                ).alias("Year, Make/Model, Cyl")).show(truncate=False)

+-------------------------------------------------------------+
|Year, Make/Model, Cyl                                        |
+-------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine                     |
|The 1999 audi a4 has a 4 cylinder engine                     |
|The 2008 audi a4 has a 4 cylinder engine                     |
|The 2008 audi a4 has a 4 cylinder engine                     |
|The 1999 audi a4 has a 6 cylinder engine                     |
|The 1999 audi a4 has a 6 cylinder engine                     |
|The 2008 audi a4 has a 6 cylinder engine                     |
|The 1999 audi a4 quattro has a 4 cylinder engine             |
|The 1999 audi a4 quattro has a 4 cylinder engine             |
|The 2008 audi a4 quattro has a 4 cylinder engine             |
|The 2008 audi a4 quattro has a 4 cylinder engine             |
|The 1999 audi a4 quattro has a 6 cylinder engine             |
|The 1999 audi a4 quattro has a 6 cylind

### Transform the trans column so that it only contains either manual or auto.

In [8]:
mpg.select(
    mpg.trans,
    (
        f.when(mpg.trans.contains("auto"), "auto")
        .otherwise("manual")
        .alias("transmission type")
    ),
).show()
print("-----------------")
mpg.select((
        f.when(mpg.trans.contains("auto"), "auto")
        .otherwise("manual")
        .alias("transmission type")
    ),
).show()

+----------+-----------------+
|     trans|transmission type|
+----------+-----------------+
|  auto(l5)|             auto|
|manual(m5)|           manual|
|manual(m6)|           manual|
|  auto(av)|             auto|
|  auto(l5)|             auto|
|manual(m5)|           manual|
|  auto(av)|             auto|
|manual(m5)|           manual|
|  auto(l5)|             auto|
|manual(m6)|           manual|
|  auto(s6)|             auto|
|  auto(l5)|             auto|
|manual(m5)|           manual|
|  auto(s6)|             auto|
|manual(m6)|           manual|
|  auto(l5)|             auto|
|  auto(s6)|             auto|
|  auto(s6)|             auto|
|  auto(l4)|             auto|
|  auto(l4)|             auto|
+----------+-----------------+
only showing top 20 rows

-----------------
+-----------------+
|transmission type|
+-----------------+
|             auto|
|           manual|
|           manual|
|             auto|
|             auto|
|           manual|
|             auto|
|           

In [9]:
#mpg.withColumn("tran",f.when(mpg.trans.like("auto%"),"auto").otherwise("manual"))

# Load the tips dataset as a spark dataframe.


In [10]:
tips = spark.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



/usr/local/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


### What percentage of observations are smokers?


In [11]:
tips.rollup("smoker").mean().show()

+------+------------------+------------------+------------------+
|smoker|   avg(total_bill)|          avg(tip)|         avg(size)|
+------+------------------+------------------+------------------+
|  null|19.785942622950813|  2.99827868852459| 2.569672131147541|
|    No|19.188278145695364|2.9918543046357615|2.6688741721854305|
|   Yes|20.756344086021503| 3.008709677419355|2.4086021505376345|
+------+------------------+------------------+------------------+



In [12]:
tips.groupby("smoker").count().withColumn("percent",f.round(f.col("count")/tips.count() *100)).show()

+------+-----+-------+
|smoker|count|percent|
+------+-----+-------+
|    No|  151|   62.0|
|   Yes|   93|   38.0|
+------+-----+-------+



In [13]:
tips.groupBy(tips.smoker).agg(f.count(tips.smoker)).show()

+------+-------------+
|smoker|count(smoker)|
+------+-------------+
|    No|          151|
|   Yes|           93|
+------+-------------+



### Create a column that contains the tip percentage


In [14]:
tips.select(f.concat(f.round((tips.tip/tips.total_bill),3)*100,f.lit("%")).alias("tip percentage")).show()

+-------------------+
|     tip percentage|
+-------------------+
|5.8999999999999995%|
|              16.1%|
|              16.7%|
|14.000000000000002%|
|              14.7%|
|              18.6%|
|              22.8%|
|11.600000000000001%|
|              13.0%|
|              21.9%|
|              16.7%|
|              14.2%|
|              10.2%|
|              16.3%|
|              20.4%|
|              18.2%|
|              16.2%|
|              22.8%|
|20.599999999999998%|
|              16.2%|
+-------------------+
only showing top 20 rows



In [15]:
#tips.withColumn("tip_percent",tips.tip/tips.total_bill).show()
#tips.select(tips.tip, tips.total_bill,f.round(tips.tip/tips.total_bill),4).alias("tip_percent").show(5)

### Calculate the average tip percentage for each combination of sex and smoker.

In [16]:
tip_p = f.concat(f.round((tips.tip/tips.total_bill),3)*100)

In [17]:
tips.groupBy(tips.smoker,tips.sex).agg(f.mean(tip_p).alias("tip percent (%)")).show()

+------+------+------------------+
|smoker|   sex|   tip percent (%)|
+------+------+------------------+
|    No|Female| 15.69259259259259|
|    No|  Male| 16.06701030927835|
|   Yes|  Male|15.281666666666666|
|   Yes|Female| 18.21515151515152|
+------+------+------------------+



In [18]:
tips.groupby("sex").pivot("smoker").agg(f.round(f.mean(tips.tip/tips.total_bill),4)).show()

+------+------+------+
|   sex|    No|   Yes|
+------+------+------+
|Female|0.1569|0.1822|
|  Male|0.1607|0.1528|
+------+------+------+



# Use the seattle weather dataset referenced in the lesson to answer the questions below.


In [19]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show()

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|
|2012-01-08|          0.0|    10.0|     2.8| 2.0|    sun|
|2012-01-09|          4.3|     9.4|     5.0| 3.4|   rain|
|2012-01-10|          1.0|     6.1|     0.6| 3.4|   rain|
|2012-01-11|          0.0|     6.1|    -1.1| 5.1|    sun|
|2012-01-12|          0.0|     6.1|    -1.7| 1.9|    sun|
|2012-01-13|          0.0|     5.0|    -2.8| 1.3|    sun|
|2012-01-14|          4.1|     4.4|     0.6| 5.3|   snow|
|2012-01-15|  

/usr/local/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


### Convert the temperatures to fahrenheit.


In [20]:
weather.temp_max = (weather.temp_max * 1.8 + 32).alias("temp_max_F")
weather.temp_min = (weather.temp_min * 1.8 + 32).alias("temp_min_F")
weather.select(weather.temp_max,weather.temp_min).show()

+------------------+------------------+
|        temp_max_F|        temp_min_F|
+------------------+------------------+
|55.040000000000006|              41.0|
|             51.08|             37.04|
|             53.06|             44.96|
|             53.96|             42.08|
|48.019999999999996|             37.04|
|             39.92|             35.96|
|             44.96|             37.04|
|              50.0|             37.04|
|             48.92|              41.0|
|42.980000000000004|             33.08|
|42.980000000000004|             30.02|
|42.980000000000004|             28.94|
|              41.0|             26.96|
|             39.92|             33.08|
|             33.98|26.060000000000002|
|             35.06|             26.96|
|             37.94|              32.0|
|              32.0|             26.96|
|             30.02|             26.96|
|             44.96|             30.02|
+------------------+------------------+
only showing top 20 rows



### Which month has the most rain, on average?


In [21]:
weather.withColumn("month",f.month("date")).groupBy("month").agg(f.mean("precipitation").alias("avg rainfall per month")).sort(col("avg rainfall per month").desc()).show(1)

+-----+----------------------+
|month|avg rainfall per month|
+-----+----------------------+
|   11|     5.354166666666667|
+-----+----------------------+
only showing top 1 row



In [22]:
weather = \
    weather.withColumn("temp_max",(col("temp_max")*9/5+32)).withColumn("temp_min",(col("temp_min")*9/5+32))
weather.show()

+----------+-------------+------------------+------------------+----+-------+
|      date|precipitation|          temp_max|          temp_min|wind|weather|
+----------+-------------+------------------+------------------+----+-------+
|2012-01-01|          0.0|             55.04|              41.0| 4.7|drizzle|
|2012-01-02|         10.9|             51.08|             37.04| 4.5|   rain|
|2012-01-03|          0.8|             53.06|             44.96| 2.3|   rain|
|2012-01-04|         20.3|             53.96|             42.08| 4.7|   rain|
|2012-01-05|          1.3|             48.02|             37.04| 6.1|   rain|
|2012-01-06|          2.5|             39.92|             35.96| 2.2|   rain|
|2012-01-07|          0.0|             44.96|             37.04| 2.3|   rain|
|2012-01-08|          0.0|              50.0|             37.04| 2.0|    sun|
|2012-01-09|          4.3|             48.92|              41.0| 3.4|   rain|
|2012-01-10|          1.0|42.980000000000004|             33.08|

In [23]:
weather.withColumn(
    "month",f.month(weather.date)
    ).groupby(
    f.col("month")
    ).agg(
        f.mean(
            weather.precipitation
            ).alias("avg_rainfall")
            ).sort(
                f.col("avg_rainfall").desc()
                ).show()

print("----------")
print(weather.withColumn(
    "month",f.month(weather.date)
    ).groupby(
    f.col("month")
    ).agg(
        f.mean(
            weather.precipitation
            ).alias("avg_rainfall")
            ).sort(
                f.col("avg_rainfall").desc()
                ).first()[0])

print("-----------")
weather.withColumn(
    "month",f.month(weather.date)
    ).groupby(
    f.col("month")
    ).agg(
        f.mean(
            weather.precipitation
            ).alias("avg_rainfall")
            ).sort(
                f.col("avg_rainfall").desc()
                ).first()

+-----+-------------------+
|month|       avg_rainfall|
+-----+-------------------+
|   11|  5.354166666666667|
|   12|  5.021774193548388|
|    3|  4.888709677419355|
|   10|  4.059677419354839|
|    1| 3.7580645161290316|
|    2|  3.734513274336283|
|    4| 3.1283333333333325|
|    9| 1.9624999999999997|
|    5| 1.6733870967741935|
|    8| 1.3201612903225806|
|    6| 1.1075000000000002|
|    7|0.38870967741935486|
+-----+-------------------+

----------
11
-----------


Row(month=11, avg_rainfall=5.354166666666667)

### Which year was the windiest?


In [24]:
weather.\
    withColumn("year",f.year("date")).\
        groupBy("year").agg(f.sum("wind").\
            alias("total wind per year")).\
                sort(col("total wind per year").\
                    desc()).show(1)

+----+-------------------+
|year|total wind per year|
+----+-------------------+
|2012| 1244.7000000000003|
+----+-------------------+
only showing top 1 row



In [25]:
weather.withColumn(
    "year",f.year(weather.date)
    ).groupby(
    f.col("year")
    ).agg(
        f.mean(
            weather.wind
            ).alias("avg_wind")
            ).sort(
                f.col("avg_wind").desc()
                ).show(1)

+----+------------------+
|year|          avg_wind|
+----+------------------+
|2012|3.4008196721311483|
+----+------------------+
only showing top 1 row



### What is the most frequent type of weather in January?


In [26]:
weather.filter(f.month("date")==1).withColumn("month",f.month("date")).groupBy("weather").count().sort(col("count").desc()).show(1)

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
+-------+-----+
only showing top 1 row



In [27]:
print("-----------")
weather.filter(
    f.month(weather.date)==1
    ).groupby(
    weather.weather
    ).count(
        ).sort(
            f.col("count").desc()
        ).show()

-----------
+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



### What is the average high and low temperature on sunny days in July in 2013 and 2014?


In [28]:
### didn't do this right
#
#weather.withColumn("year", f.year("date"))\
#    #.filter(f.year("date")==2013,2014)\
#    .withColumn("month", f.month("date"))\
#    .filter(f.month("date")==7)\
#    .groupBy("month","weather")\
#    .agg(f.mean("temp_min"),f.mean("temp_max"))\
#    .filter(col("weather")=="sun")\
#    .show()

In [29]:
#weather.filter(f.month("date") == 7)
#    .filter(f.year("date") > 2012)
#    .filter(f.year("date")<2015)

### What percentage of days were rainy in q3 of 2015?


In [30]:
#total_observations = weather.withColumn("year", f.year("date"))\
#    .filter(col("year")==2015)\
#    .withColumn("quarter", f.quarter("date"))\
#    .filter(f.quarter("date")==3)
#
#weather.withColumn("year", f.year("date"))\
#    .filter(col("year")==2015)\
#    .withColumn("quarter", f.quarter("date"))\
#    .filter(f.quarter("date")==3)\
#    .rollup("weather")\
#    .count()\
#    .withColumn("count",f.col("count")/total_observations)\
#    .show()
#
#    #.withColumn("month", f.month("date"))\
#    #.filter(f.month("date")==7)\
#    #.groupBy("month","weather")\
#    #.agg(f.mean("temp_min"),f.mean("temp_max"))\
#    #.filter(col("weather")=="sun")\
#    #.show()

In [31]:
weather.filter(f.year('date') == 2015)\
.filter(f.quarter('date')==3)\
.select(f.when(f.col("weather")=="rain",1).otherwise((0)).alias("rain"))\
.agg(f.mean('rain'))\
.show()

+--------------------+
|           avg(rain)|
+--------------------+
|0.021739130434782608|
+--------------------+



### For each year, find what percentage of days it rained (had non-zero precipitation).

In [32]:
weather.withColumn(\
    "year", f.year("date")
)\
.select(
        f.when(
        f.col("precipitation")>0,1
        ).otherwise(0).alias("did_rain"),"year"
        )\
.groupby("year")\
.agg(f.mean("did_rain"))\
.show()


+----+-------------------+
|year|      avg(did_rain)|
+----+-------------------+
|2012|0.48360655737704916|
|2013|0.41643835616438357|
|2014|  0.410958904109589|
|2015|0.39452054794520547|
+----+-------------------+



------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------
------

In [33]:
spark

In [34]:
import pandas as pd
import numpy as np

np.random.seed(456)

pandas_dataframe = pd.DataFrame(
    dict(n=np.arange(20), group=np.random.choice(list("abc"), 20))
)
pandas_dataframe


,n,group
0,0,b
1,1,b
2,2,c
3,3,a
4,4,c
5,5,c
6,6,a
7,7,b
8,8,a
9,9,b


In [35]:
df = spark.createDataFrame(pandas_dataframe)
df


/usr/local/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


DataFrame[n: bigint, group: string]

In [36]:
df.show(5)


+---+-----+
|  n|group|
+---+-----+
|  0|    b|
|  1|    b|
|  2|    c|
|  3|    a|
|  4|    c|
+---+-----+
only showing top 5 rows



In [37]:
df.describe()


DataFrame[summary: string, n: string, group: string]

In [38]:
df.describe().show()


+-------+-----------------+-----+
|summary|                n|group|
+-------+-----------------+-----+
|  count|               20|   20|
|   mean|              9.5| null|
| stddev|5.916079783099616| null|
|    min|                0|    a|
|    max|               19|    c|
+-------+-----------------+-----+



In [39]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)


+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



/usr/local/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [40]:
mpg.hwy


Column<'hwy'>

In [41]:
mpg.select(mpg.hwy, mpg.cty, mpg.model)


DataFrame[hwy: bigint, cty: bigint, model: string]

In [42]:
mpg.select(mpg.hwy, mpg.cty, mpg.model).show(10)


+---+---+----------+
|hwy|cty|     model|
+---+---+----------+
| 29| 18|        a4|
| 29| 21|        a4|
| 31| 20|        a4|
| 30| 21|        a4|
| 26| 16|        a4|
| 26| 18|        a4|
| 27| 18|        a4|
| 26| 18|a4 quattro|
| 25| 16|a4 quattro|
| 28| 20|a4 quattro|
+---+---+----------+
only showing top 10 rows



|---|---|----------|
|hwy|cty|     model|
|---|---|----------|
| 29| 18|        a4|
| 29| 21|        a4|
| 31| 20|        a4|
| 30| 21|        a4|
| 26| 16|        a4|
| 26| 18|        a4|
| 27| 18|        a4|
| 26| 18|a4 quattro|
| 25| 16|a4 quattro|
| 28| 20|a4 quattro|
|---|---|----------|
only showing top 10 rows


In [43]:
mpg.hwy + 1


Column<'(hwy + 1)'>

In [44]:
mpg.select(mpg.hwy, mpg.hwy + 1).show(5)


+---+---------+
|hwy|(hwy + 1)|
+---+---------+
| 29|       30|
| 29|       30|
| 31|       32|
| 30|       31|
| 26|       27|
+---+---------+
only showing top 5 rows



In [45]:
mpg.select(mpg.hwy.alias("highway_mileage")).show(5)


+---------------+
|highway_mileage|
+---------------+
|             29|
|             29|
|             31|
|             30|
|             26|
+---------------+
only showing top 5 rows



In [46]:
col1 = mpg.hwy.alias("highway_mileage")
col2 = (mpg.hwy / 2).alias("highway_mileage_halved")
mpg.select(col1, col2).show(5)


+---------------+----------------------+
|highway_mileage|highway_mileage_halved|
+---------------+----------------------+
|             29|                  14.5|
|             29|                  14.5|
|             31|                  15.5|
|             30|                  15.0|
|             26|                  13.0|
+---------------+----------------------+
only showing top 5 rows



In [47]:
mpg.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

In [48]:
from pyspark.sql.functions import col, expr
col("hwy")



Column<'hwy'>

In [49]:
avg_column = (col("hwy") + col("cty")) / 2

mpg.select(
    col("hwy").alias("highway_mileage"),
    mpg.cty.alias("city_mileage"),
    avg_column.alias("avg_mileage"),
).show(5)


+---------------+------------+-----------+
|highway_mileage|city_mileage|avg_mileage|
+---------------+------------+-----------+
|             29|          18|       23.5|
|             29|          21|       25.0|
|             31|          20|       25.5|
|             30|          21|       25.5|
|             26|          16|       21.0|
+---------------+------------+-----------+
only showing top 5 rows



In [50]:
mpg.select(
    expr("hwy"),  # the same as `col`
    expr("hwy + 1"),  # an arithmetic expression
    expr("hwy AS highway_mileage"),  # using an alias
    expr("hwy + 1 AS highway_incremented"),  # a combination of the above
).show(5)


+---+---------+---------------+-------------------+
|hwy|(hwy + 1)|highway_mileage|highway_incremented|
+---+---------+---------------+-------------------+
| 29|       30|             29|                 30|
| 29|       30|             29|                 30|
| 31|       32|             31|                 32|
| 30|       31|             30|                 31|
| 26|       27|             26|                 27|
+---+---------+---------------+-------------------+
only showing top 5 rows



In [51]:
mpg.select(
    mpg.hwy.alias("highway"),
    col("hwy").alias("highway"),
    expr("hwy").alias("highway"),
    expr("hwy AS highway"),
).show(5)


+-------+-------+-------+-------+
|highway|highway|highway|highway|
+-------+-------+-------+-------+
|     29|     29|     29|     29|
|     29|     29|     29|     29|
|     31|     31|     31|     31|
|     30|     30|     30|     30|
|     26|     26|     26|     26|
+-------+-------+-------+-------+
only showing top 5 rows



In [52]:
mpg.createOrReplaceTempView("mpg")


In [53]:
spark.sql(
    """
SELECT hwy, cty, (hwy + cty) / 2 AS avg
FROM mpg
"""
)


DataFrame[hwy: bigint, cty: bigint, avg: double]

In [54]:
spark.sql(
    """
SELECT hwy, cty, (hwy + cty) / 2 AS avg
FROM mpg
"""
).show(5)


+---+---+----+
|hwy|cty| avg|
+---+---+----+
| 29| 18|23.5|
| 29| 21|25.0|
| 31| 20|25.5|
| 30| 21|25.5|
| 26| 16|21.0|
+---+---+----+
only showing top 5 rows



In [55]:
mpg.dtypes


[('manufacturer', 'string'),
 ('model', 'string'),
 ('displ', 'double'),
 ('year', 'bigint'),
 ('cyl', 'bigint'),
 ('trans', 'string'),
 ('drv', 'string'),
 ('cty', 'bigint'),
 ('hwy', 'bigint'),
 ('fl', 'string'),
 ('class', 'string')]

In [56]:
mpg.printSchema()


root
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- displ: double (nullable = true)
 |-- year: long (nullable = true)
 |-- cyl: long (nullable = true)
 |-- trans: string (nullable = true)
 |-- drv: string (nullable = true)
 |-- cty: long (nullable = true)
 |-- hwy: long (nullable = true)
 |-- fl: string (nullable = true)
 |-- class: string (nullable = true)



In [57]:
mpg.select(mpg.hwy.cast("string")).printSchema()


root
 |-- hwy: string (nullable = true)



In [58]:
mpg.select(mpg.model, mpg.model.cast("int")).show(5)


+-----+-----+
|model|model|
+-----+-----+
|   a4| null|
|   a4| null|
|   a4| null|
|   a4| null|
|   a4| null|
+-----+-----+
only showing top 5 rows



In [59]:
# Note: The pyspark avg and mean functions are aliases of each other
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean

from pyspark.sql.functions import *

### these functions are done on the query and not locally
# import pyspark.sql.functions as f
# f.sum() f.avg()

In [60]:
mpg.select(
    sum(mpg.hwy) / count(mpg.hwy).alias("average_1"),
    avg(mpg.hwy).alias("average_2"),
    min(mpg.hwy),
    max(mpg.hwy),
).show()


+------------------------------------+-----------------+--------+--------+
|(sum(hwy) / count(hwy) AS average_1)|        average_2|min(hwy)|max(hwy)|
+------------------------------------+-----------------+--------+--------+
|                   23.44017094017094|23.44017094017094|      12|      44|
+------------------------------------+-----------------+--------+--------+



In [61]:
mpg.select(concat(mpg.manufacturer, lit(" - "), mpg.model)).show(5)

+--------------------------------+
|concat(manufacturer,  - , model)|
+--------------------------------+
|                       audi - a4|
|                       audi - a4|
|                       audi - a4|
|                       audi - a4|
|                       audi - a4|
+--------------------------------+
only showing top 5 rows



In [62]:
from pyspark.sql.functions import regexp_extract, regexp_replace

In [63]:
textdf = spark.createDataFrame(
    pd.DataFrame(
        {
            "address": [
                "600 Navarro St ste 600, San Antonio, TX 78205",
                "3130 Broadway St, San Antonio, TX 78209",
                "303 Pearl Pkwy, San Antonio, TX 78215",
                "1255 SW Loop 410, San Antonio, TX 78227",
            ]
        }
    )
)

textdf.show(truncate=False)


+---------------------------------------------+
|address                                      |
+---------------------------------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|
|3130 Broadway St, San Antonio, TX 78209      |
|303 Pearl Pkwy, San Antonio, TX 78215        |
|1255 SW Loop 410, San Antonio, TX 78227      |
+---------------------------------------------+



In [64]:
textdf.select(
    "address",
    regexp_extract("address", r"^(\d+)", 1).alias("street_no"),
    regexp_extract("address", r"^\d+\s([\w\s]+?),", 1).alias("street"),
).show(truncate=False)


+---------------------------------------------+---------+------------------+
|address                                      |street_no|street            |
+---------------------------------------------+---------+------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|600      |Navarro St ste 600|
|3130 Broadway St, San Antonio, TX 78209      |3130     |Broadway St       |
|303 Pearl Pkwy, San Antonio, TX 78215        |303      |Pearl Pkwy        |
|1255 SW Loop 410, San Antonio, TX 78227      |1255     |SW Loop 410       |
+---------------------------------------------+---------+------------------+



In [65]:
textdf.select(
    "address",
    regexp_replace("address", r"^.*?,\s*", "").alias("city_state_zip"),
).show(truncate=False)


+---------------------------------------------+---------------------+
|address                                      |city_state_zip       |
+---------------------------------------------+---------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|San Antonio, TX 78205|
|3130 Broadway St, San Antonio, TX 78209      |San Antonio, TX 78209|
|303 Pearl Pkwy, San Antonio, TX 78215        |San Antonio, TX 78215|
|1255 SW Loop 410, San Antonio, TX 78227      |San Antonio, TX 78227|
+---------------------------------------------+---------------------+



In [66]:
mpg.filter(mpg.cyl == 4).where(mpg["class"] == "subcompact").show()


+------------+-----------+-----+----+---+----------+---+---+---+---+----------+
|manufacturer|      model|displ|year|cyl|     trans|drv|cty|hwy| fl|     class|
+------------+-----------+-----+----+---+----------+---+---+---+---+----------+
|       honda|      civic|  1.6|1999|  4|manual(m5)|  f| 28| 33|  r|subcompact|
|       honda|      civic|  1.6|1999|  4|  auto(l4)|  f| 24| 32|  r|subcompact|
|       honda|      civic|  1.6|1999|  4|manual(m5)|  f| 25| 32|  r|subcompact|
|       honda|      civic|  1.6|1999|  4|manual(m5)|  f| 23| 29|  p|subcompact|
|       honda|      civic|  1.6|1999|  4|  auto(l4)|  f| 24| 32|  r|subcompact|
|       honda|      civic|  1.8|2008|  4|manual(m5)|  f| 26| 34|  r|subcompact|
|       honda|      civic|  1.8|2008|  4|  auto(l5)|  f| 25| 36|  r|subcompact|
|       honda|      civic|  1.8|2008|  4|  auto(l5)|  f| 24| 36|  c|subcompact|
|       honda|      civic|  2.0|2008|  4|manual(m6)|  f| 21| 29|  p|subcompact|
|     hyundai|    tiburon|  2.0|1999|  4

In [67]:
from pyspark.sql.functions import when


In [68]:
mpg.select(mpg.hwy, when(mpg.hwy > 25, "good_mileage").alias("mpg_desc")).show(
    12
)


+---+------------+
|hwy|    mpg_desc|
+---+------------+
| 29|good_mileage|
| 29|good_mileage|
| 31|good_mileage|
| 30|good_mileage|
| 26|good_mileage|
| 26|good_mileage|
| 27|good_mileage|
| 26|good_mileage|
| 25|        null|
| 28|good_mileage|
| 27|good_mileage|
| 25|        null|
+---+------------+
only showing top 12 rows



In [69]:
mpg.select(
    mpg.hwy,
    when(mpg.hwy > 25, "good_mileage")
    .otherwise("bad_mileage")
    .alias("mpg_desc"),
).show(12)


+---+------------+
|hwy|    mpg_desc|
+---+------------+
| 29|good_mileage|
| 29|good_mileage|
| 31|good_mileage|
| 30|good_mileage|
| 26|good_mileage|
| 26|good_mileage|
| 27|good_mileage|
| 26|good_mileage|
| 25| bad_mileage|
| 28|good_mileage|
| 27|good_mileage|
| 25| bad_mileage|
+---+------------+
only showing top 12 rows



In [70]:
mpg.select(
    mpg.displ,
    (
        when(mpg.displ < 2, "small")
        .when(mpg.displ < 3, "medium")
        .otherwise("large")
        .alias("engine_size")
    ),
).show(10)


+-----+-----------+
|displ|engine_size|
+-----+-----------+
|  1.8|      small|
|  1.8|      small|
|  2.0|     medium|
|  2.0|     medium|
|  2.8|     medium|
|  2.8|     medium|
|  3.1|      large|
|  1.8|      small|
|  1.8|      small|
|  2.0|     medium|
+-----+-----------+
only showing top 10 rows



In [71]:
mpg.sort(mpg.hwy).show(8)


+------------+-------------------+-----+----+---+----------+---+---+---+---+------+
|manufacturer|              model|displ|year|cyl|     trans|drv|cty|hwy| fl| class|
+------------+-------------------+-----+----+---+----------+---+---+---+---+------+
|       dodge|ram 1500 pickup 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|pickup|
|       dodge|ram 1500 pickup 4wd|  4.7|2008|  8|manual(m6)|  4|  9| 12|  e|pickup|
|       dodge|  dakota pickup 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|pickup|
|        jeep| grand cherokee 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|   suv|
|       dodge|        durango 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|   suv|
|   chevrolet|    k1500 tahoe 4wd|  5.3|2008|  8|  auto(l4)|  4| 11| 14|  e|   suv|
|        jeep| grand cherokee 4wd|  6.1|2008|  8|  auto(l5)|  4| 11| 14|  p|   suv|
|  land rover|        range rover|  4.0|1999|  8|  auto(l4)|  4| 11| 15|  p|   suv|
+------------+-------------------+-----+----+---+----------+---+---+---+---+

In [72]:
from pyspark.sql.functions import asc, desc


In [73]:
mpg.sort(mpg.hwy.desc())
# is the same as
mpg.sort(col("hwy").desc())
# is the same as
mpg.sort(desc("hwy")).show(5)


+------------+----------+-----+----+---+----------+---+---+---+---+----------+
|manufacturer|     model|displ|year|cyl|     trans|drv|cty|hwy| fl|     class|
+------------+----------+-----+----+---+----------+---+---+---+---+----------+
|  volkswagen|     jetta|  1.9|1999|  4|manual(m5)|  f| 33| 44|  d|   compact|
|  volkswagen|new beetle|  1.9|1999|  4|manual(m5)|  f| 35| 44|  d|subcompact|
|  volkswagen|new beetle|  1.9|1999|  4|  auto(l4)|  f| 29| 41|  d|subcompact|
|      toyota|   corolla|  1.8|2008|  4|manual(m5)|  f| 28| 37|  r|   compact|
|       honda|     civic|  1.8|2008|  4|  auto(l5)|  f| 25| 36|  r|subcompact|
+------------+----------+-----+----+---+----------+---+---+---+---+----------+
only showing top 5 rows



In [74]:
mpg.sort(desc("class"), mpg.cyl.asc(), col("hwy").desc()).show()


+------------+------------------+-----+----+---+----------+---+---+---+---+-----+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-----+
|      subaru|      forester awd|  2.5|2008|  4|manual(m5)|  4| 20| 27|  r|  suv|
|      subaru|      forester awd|  2.5|2008|  4|  auto(l4)|  4| 20| 26|  r|  suv|
|      subaru|      forester awd|  2.5|2008|  4|manual(m5)|  4| 19| 25|  p|  suv|
|      subaru|      forester awd|  2.5|1999|  4|manual(m5)|  4| 18| 25|  r|  suv|
|      subaru|      forester awd|  2.5|1999|  4|  auto(l4)|  4| 18| 24|  r|  suv|
|      subaru|      forester awd|  2.5|2008|  4|  auto(l4)|  4| 18| 23|  p|  suv|
|      toyota|       4runner 4wd|  2.7|1999|  4|  auto(l4)|  4| 16| 20|  r|  suv|
|      toyota|       4runner 4wd|  2.7|1999|  4|manual(m5)|  4| 15| 20|  r|  suv|
|        jeep|grand cherokee 4wd|  3.0|2008|  6|  auto(l5)|  4| 17| 22|  d|  suv|
|      toyota|  

In [75]:
mpg.groupBy(mpg.cyl)
mpg.groupBy(col("cyl"))
mpg.groupBy("cyl")

mpg.groupBy(mpg.cyl).agg(avg(mpg.cty), avg(mpg.hwy)).show()

mpg.groupBy()

+---+------------------+-----------------+
|cyl|          avg(cty)|         avg(hwy)|
+---+------------------+-----------------+
|  6| 16.21518987341772|22.82278481012658|
|  8|12.571428571428571|17.62857142857143|
|  4|21.012345679012345|28.80246913580247|
|  5|              20.5|            28.75|
+---+------------------+-----------------+



In [76]:
mpg.rollup("cyl").count().sort("cyl").show()


+----+-----+
| cyl|count|
+----+-----+
|null|  234|
|   4|   81|
|   5|    4|
|   6|   79|
|   8|   70|
+----+-----+



In [77]:
mpg.rollup("cyl").agg(expr("avg(hwy)")).sort("cyl").show()


+----+-----------------+
| cyl|         avg(hwy)|
+----+-----------------+
|null|23.44017094017094|
|   4|28.80246913580247|
|   5|            28.75|
|   6|22.82278481012658|
|   8|17.62857142857143|
+----+-----------------+



In [78]:
mpg.rollup("cyl", "class").mean("hwy").sort(col("cyl"), col("class")).show()


+----+----------+------------------+
| cyl|     class|          avg(hwy)|
+----+----------+------------------+
|null|      null| 23.44017094017094|
|   4|      null| 28.80246913580247|
|   4|   compact|          29.46875|
|   4|   midsize|           29.1875|
|   4|   minivan|              24.0|
|   4|    pickup|20.666666666666668|
|   4|subcompact| 30.80952380952381|
|   4|       suv|             23.75|
|   5|      null|             28.75|
|   5|   compact|              29.0|
|   5|subcompact|              28.5|
|   6|      null| 22.82278481012658|
|   6|   compact|25.307692307692307|
|   6|   midsize| 26.26086956521739|
|   6|   minivan|              22.2|
|   6|    pickup|              17.9|
|   6|subcompact|24.714285714285715|
|   6|       suv|              18.5|
|   8|      null| 17.62857142857143|
|   8|   2seater|              24.8|
+----+----------+------------------+
only showing top 20 rows



In [79]:
mpg.rollup("cyl").agg(expr("avg(hwy)")).sort("cyl").show()


+----+-----------------+
| cyl|         avg(hwy)|
+----+-----------------+
|null|23.44017094017094|
|   4|28.80246913580247|
|   5|            28.75|
|   6|22.82278481012658|
|   8|17.62857142857143|
+----+-----------------+



In [80]:
mpg.crosstab("class", "cyl").show()


+----------+---+---+---+---+
| class_cyl|  4|  5|  6|  8|
+----------+---+---+---+---+
|   midsize| 16|  0| 23|  2|
|subcompact| 21|  2|  7|  5|
|   2seater|  0|  0|  0|  5|
|    pickup|  3|  0| 10| 20|
|   minivan|  1|  0| 10|  0|
|       suv|  8|  0| 16| 38|
|   compact| 32|  2| 13|  0|
+----------+---+---+---+---+



In [81]:
mpg.groupby("class").pivot("cyl").mean("hwy").show()


+----------+------------------+----+------------------+------------------+
|     class|                 4|   5|                 6|                 8|
+----------+------------------+----+------------------+------------------+
|subcompact| 30.80952380952381|28.5|24.714285714285715|              21.6|
|   compact|          29.46875|29.0|25.307692307692307|              null|
|   minivan|              24.0|null|              22.2|              null|
|       suv|             23.75|null|              18.5|16.789473684210527|
|   midsize|           29.1875|null| 26.26086956521739|              24.0|
|    pickup|20.666666666666668|null|              17.9|              15.8|
|   2seater|              null|null|              null|              24.8|
+----------+------------------+----+------------------+------------------+



In [82]:
df = spark.createDataFrame(
    pd.DataFrame(
        {"x": [1, 2, np.nan, 4, 5, np.nan], "y": [np.nan, 0, 0, 3, 1, np.nan]}
    )
)
df.show()


+---+---+
|  x|  y|
+---+---+
|1.0|NaN|
|2.0|0.0|
|NaN|0.0|
|4.0|3.0|
|5.0|1.0|
|NaN|NaN|
+---+---+



In [83]:
df.na.drop().show()


+---+---+
|  x|  y|
+---+---+
|2.0|0.0|
|4.0|3.0|
|5.0|1.0|
+---+---+



In [84]:
df.na.fill(0).show()


+---+---+
|  x|  y|
+---+---+
|1.0|0.0|
|2.0|0.0|
|0.0|0.0|
|4.0|3.0|
|5.0|1.0|
|0.0|0.0|
+---+---+



In [85]:
df.na.fill(0, subset="x").show()


+---+---+
|  x|  y|
+---+---+
|1.0|NaN|
|2.0|0.0|
|0.0|0.0|
|4.0|3.0|
|5.0|1.0|
|0.0|NaN|
+---+---+



In [86]:
df.na.drop(subset="y").show()


+---+---+
|  x|  y|
+---+---+
|2.0|0.0|
|NaN|0.0|
|4.0|3.0|
|5.0|1.0|
+---+---+



In [87]:
mpg.explain()


== Physical Plan ==
*(1) Scan ExistingRDD[manufacturer#1193,model#1194,displ#1195,year#1196L,cyl#1197L,trans#1198,drv#1199,cty#1200L,hwy#1201L,fl#1202,class#1203]




In [88]:
mpg.select(mpg.cyl, mpg.hwy).explain()


== Physical Plan ==
*(1) Project [cyl#1197L, hwy#1201L]
+- *(1) Scan ExistingRDD[manufacturer#1193,model#1194,displ#1195,year#1196L,cyl#1197L,trans#1198,drv#1199,cty#1200L,hwy#1201L,fl#1202,class#1203]




In [89]:
mpg.select(((mpg.cyl + mpg.hwy) / 2).alias("avg_mpg")).explain()


== Physical Plan ==
*(1) Project [(cast((cyl#1197L + hwy#1201L) as double) / 2.0) AS avg_mpg#2290]
+- *(1) Scan ExistingRDD[manufacturer#1193,model#1194,displ#1195,year#1196L,cyl#1197L,trans#1198,drv#1199,cty#1200L,hwy#1201L,fl#1202,class#1203]




In [90]:
mpg.filter(mpg.cyl == 6).explain()


== Physical Plan ==
*(1) Filter (isnotnull(cyl#1197L) AND (cyl#1197L = 6))
+- *(1) Scan ExistingRDD[manufacturer#1193,model#1194,displ#1195,year#1196L,cyl#1197L,trans#1198,drv#1199,cty#1200L,hwy#1201L,fl#1202,class#1203]




In [91]:
mpg.select("cyl", "hwy").filter(expr("cyl = 6")).explain()
mpg.filter(expr("cyl = 6")).select("cyl", "hwy").explain()


== Physical Plan ==
*(1) Project [cyl#1197L, hwy#1201L]
+- *(1) Filter (isnotnull(cyl#1197L) AND (cyl#1197L = 6))
   +- *(1) Scan ExistingRDD[manufacturer#1193,model#1194,displ#1195,year#1196L,cyl#1197L,trans#1198,drv#1199,cty#1200L,hwy#1201L,fl#1202,class#1203]


== Physical Plan ==
*(1) Project [cyl#1197L, hwy#1201L]
+- *(1) Filter (isnotnull(cyl#1197L) AND (cyl#1197L = 6))
   +- *(1) Scan ExistingRDD[manufacturer#1193,model#1194,displ#1195,year#1196L,cyl#1197L,trans#1198,drv#1199,cty#1200L,hwy#1201L,fl#1202,class#1203]




In [92]:
mpg.selectExpr("cyl + 3 * 16 / 4 + 19 AS unused", "hwy").select(
    "hwy"
).explain()


== Physical Plan ==
*(1) Project [hwy#1201L]
+- *(1) Scan ExistingRDD[manufacturer#1193,model#1194,displ#1195,year#1196L,cyl#1197L,trans#1198,drv#1199,cty#1200L,hwy#1201L,fl#1202,class#1203]




In [93]:
mpg.select(min(mpg.cyl)).explain()


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[min(cyl#1197L)])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#2060]
      +- HashAggregate(keys=[], functions=[partial_min(cyl#1197L)])
         +- Project [cyl#1197L]
            +- Scan ExistingRDD[manufacturer#1193,model#1194,displ#1195,year#1196L,cyl#1197L,trans#1198,drv#1199,cty#1200L,hwy#1201L,fl#1202,class#1203]




In [94]:
(
    mpg.select(col("cyl"), expr("(cty + hwy) / 2 AS avg_mpg"))
    .filter(expr('class == "compact"'))
    .groupby("cyl")
    .agg(min("avg_mpg"), avg("avg_mpg"), max("avg_mpg"))
    .explain()
)


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[cyl#1197L], functions=[min(avg_mpg#2305), avg(avg_mpg#2305), max(avg_mpg#2305)])
   +- Exchange hashpartitioning(cyl#1197L, 200), ENSURE_REQUIREMENTS, [id=#2081]
      +- HashAggregate(keys=[cyl#1197L], functions=[partial_min(avg_mpg#2305), partial_avg(avg_mpg#2305), partial_max(avg_mpg#2305)])
         +- Project [cyl#1197L, (cast((cty#1200L + hwy#1201L) as double) / 2.0) AS avg_mpg#2305]
            +- Filter (isnotnull(class#1203) AND (class#1203 = compact))
               +- Scan ExistingRDD[manufacturer#1193,model#1194,displ#1195,year#1196L,cyl#1197L,trans#1198,drv#1199,cty#1200L,hwy#1201L,fl#1202,class#1203]




In [95]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)


+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



/usr/local/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [96]:
print(weather.count(), "rows", len(weather.columns), "columns")


1461 rows 6 columns


In [97]:
min_date, max_date = weather.select(min("date"), max("date")).first()
min_date, max_date


('2012-01-01', '2015-12-31')

In [98]:
weather = weather.withColumn(
    "temp_avg", expr("ROUND(temp_min + temp_max) / 2")
).drop("temp_max", "temp_min")
weather.show(6)


+----------+-------------+----+-------+--------+
|      date|precipitation|wind|weather|temp_avg|
+----------+-------------+----+-------+--------+
|2012-01-01|          0.0| 4.7|drizzle|     9.0|
|2012-01-02|         10.9| 4.5|   rain|     6.5|
|2012-01-03|          0.8| 2.3|   rain|     9.5|
|2012-01-04|         20.3| 4.7|   rain|     9.0|
|2012-01-05|          1.3| 6.1|   rain|     6.0|
|2012-01-06|          2.5| 2.2|   rain|     3.5|
+----------+-------------+----+-------+--------+
only showing top 6 rows



In [99]:
from pyspark.sql.functions import month, year, quarter


In [100]:
(
    weather.withColumn("month", month("date"))
    .groupBy("month")
    .agg(sum("precipitation").alias("total_rainfall"))
    .sort("month")
    .show()
)


+-----+------------------+
|month|    total_rainfall|
+-----+------------------+
|    1|465.99999999999994|
|    2|             422.0|
|    3|             606.2|
|    4| 375.3999999999999|
|    5|             207.5|
|    6|             132.9|
|    7|              48.2|
|    8|             163.7|
|    9|235.49999999999997|
|   10|             503.4|
|   11|             642.5|
|   12|             622.7|
+-----+------------------+



In [101]:
(
    weather.filter(month("date") == 12)
    .filter(year("date") == 2013)
    .groupBy("weather")
    .agg(mean("temp_avg"))
    .show()
)


+-------+-----------------+
|weather|    avg(temp_avg)|
+-------+-----------------+
|    fog|7.555555555555555|
|    sun|2.977272727272727|
+-------+-----------------+



In [102]:
(
    weather.filter(year("date") == 2013)
    .withColumn("freezing_temps", (weather.temp_avg <= 0).cast("int"))
    .withColumn("month", month("date"))
    .groupBy("month")
    .agg(sum("freezing_temps").alias("no_of_days_with_freezing_temps"))
    .sort("month")
    .show()
)


+-----+------------------------------+
|month|no_of_days_with_freezing_temps|
+-----+------------------------------+
|    1|                             3|
|    2|                             0|
|    3|                             0|
|    4|                             0|
|    5|                             0|
|    6|                             0|
|    7|                             0|
|    8|                             0|
|    9|                             0|
|   10|                             0|
|   11|                             0|
|   12|                             5|
+-----+------------------------------+



In [103]:
(
    weather.withColumn("quarter", quarter("date"))
    .withColumn("year", year("date"))
    .groupBy("year", "quarter")
    .agg(mean("temp_avg").alias("temp_avg"))
    .sort("year", "quarter")
    .show()
)


+----+-------+------------------+
|year|quarter|          temp_avg|
+----+-------+------------------+
|2012|      1| 5.587912087912088|
|2012|      2|12.675824175824175|
|2012|      3|            18.375|
|2012|      4| 8.581521739130435|
|2013|      1| 6.405555555555556|
|2013|      2|14.505494505494505|
|2013|      3| 19.47826086956522|
|2013|      4| 8.032608695652174|
|2014|      1| 7.205555555555556|
|2014|      2|14.296703296703297|
|2014|      3|19.858695652173914|
|2014|      4|  9.88586956521739|
|2015|      1| 8.972222222222221|
|2015|      2|15.258241758241759|
|2015|      3|19.407608695652176|
|2015|      4| 8.956521739130435|
+----+-------+------------------+



In [104]:
(
    weather.withColumn("quarter", quarter("date"))
    .withColumn("year", year("date"))
    .groupBy("quarter")
    .pivot("year")
    .agg(expr("ROUND(MEAN(temp_avg), 2) AS temp_avg"))
    .sort("quarter")
    .show()
)


+-------+-----+-----+-----+-----+
|quarter| 2012| 2013| 2014| 2015|
+-------+-----+-----+-----+-----+
|      1| 5.59| 6.41| 7.21| 8.97|
|      2|12.68|14.51| 14.3|15.26|
|      3|18.38|19.48|19.86|19.41|
|      4| 8.58| 8.03| 9.89| 8.96|
+-------+-----+-----+-----+-----+



In [105]:
users = spark.createDataFrame(
    pd.DataFrame(
        {
            "id": [1, 2, 3, 4, 5, 6],
            "name": ["bob", "joe", "sally", "adam", "jane", "mike"],
            "role_id": [1, 2, 3, 3, np.nan, np.nan],
        }
    )
)
roles = spark.createDataFrame(
    pd.DataFrame(
        {
            "id": [1, 2, 3, 4],
            "name": ["admin", "author", "reviewer", "commenter"],
        }
    )
)
print("--- users ---")
users.show()
print("--- roles ---")
roles.show()


--- users ---
+---+-----+-------+
| id| name|role_id|
+---+-----+-------+
|  1|  bob|    1.0|
|  2|  joe|    2.0|
|  3|sally|    3.0|
|  4| adam|    3.0|
|  5| jane|    NaN|
|  6| mike|    NaN|
+---+-----+-------+

--- roles ---
+---+---------+
| id|     name|
+---+---------+
|  1|    admin|
|  2|   author|
|  3| reviewer|
|  4|commenter|
+---+---------+



In [106]:
users.join(roles, on=users.role_id == roles.id).show()


+---+-----+-------+---+--------+
| id| name|role_id| id|    name|
+---+-----+-------+---+--------+
|  1|  bob|    1.0|  1|   admin|
|  2|  joe|    2.0|  2|  author|
|  3|sally|    3.0|  3|reviewer|
|  4| adam|    3.0|  3|reviewer|
+---+-----+-------+---+--------+



In [107]:
users.join(roles, on=users.role_id == roles.id, how="left").show()


+---+-----+-------+----+--------+
| id| name|role_id|  id|    name|
+---+-----+-------+----+--------+
|  1|  bob|    1.0|   1|   admin|
|  2|  joe|    2.0|   2|  author|
|  3|sally|    3.0|   3|reviewer|
|  4| adam|    3.0|   3|reviewer|
|  5| jane|    NaN|null|    null|
|  6| mike|    NaN|null|    null|
+---+-----+-------+----+--------+



In [108]:
users.join(roles, on=users.role_id == roles.id, how="right").show()


+----+-----+-------+---+---------+
|  id| name|role_id| id|     name|
+----+-----+-------+---+---------+
|   1|  bob|    1.0|  1|    admin|
|   2|  joe|    2.0|  2|   author|
|   4| adam|    3.0|  3| reviewer|
|   3|sally|    3.0|  3| reviewer|
|null| null|   null|  4|commenter|
+----+-----+-------+---+---------+

